In [1226]:
import pandas as pd
import numpy as np
from sklearn.cluster import HDBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist, pdist

## Importazione dati di lavoro
Questa sezione andrà riscritta quando faremo i collegamenti client-server e da/verso il database

### Importiamo i dati
Qui sarà la parte dove importiamo i dati dal database

In [ ]:
df1 = pd.read_csv("dataset2.csv")
unnecessary = ["name", "bio", "gender", "gender_other", "age_flag_other",
                   "age_radius_other", "distance_flag_other", "distance_km_other",
                   "same_interests"]
df = df1.drop(columns=unnecessary)
df.describe()

,ID,age,longitude,latitude,attractiveness,sincerity,intelligence,funniness,ambition,sports,...,hiking_other,gaming_other,clubbing_other,reading_other,tv_other,theater_other,movies_other,music_other,shopping_other,yoga_other
count,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,...,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000
mean,4158.508547,26.559829,10.269286,44.846729,7.123932,7.354701,5.893162,6.393162,6.175214,5.572650,...,5.435897,4.029915,5.470085,4.683761,5.273504,6.119658,5.679487,5.803419,5.777778,5.350427
std,2459.824869,3.684827,0.499163,0.355461,1.141127,0.882738,1.377827,0.993172,1.204099,2.041695,...,2.112235,1.634907,2.059512,1.756164,1.983295,1.941914,2.080939,1.694421,1.923923,2.095637
min,11.000000,21.000000,9.256040,44.098900,3.000000,2.000000,1.000000,3.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,1961.500000,24.000000,9.906305,44.572875,7.000000,7.000000,5.000000,6.000000,6.000000,4.000000,...,3.000000,3.000000,3.250000,4.000000,3.000000,5.000000,4.000000,4.000000,4.000000,3.000000
50%,4056.000000,26.000000,10.273850,44.933250,7.000000,8.000000,6.000000,6.000000,6.000000,6.000000,...,5.000000,4.000000,6.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,6.000000
75%,6170.750000,28.750000,10.739300,45.153800,8.000000,8.000000,7.000000,7.000000,7.000000,7.000000,...,7.000000,5.000000,7.000000,6.000000,7.000000,8.000000,8.000000,7.000000,7.000000,7.000000
max,8593.000000,38.000000,11.014200,45.350000,9.000000,8.000000,8.000000,8.000000,8.000000,9.000000,...,9.000000,7.000000,9.000000,7.000000,8.000000,9.000000,8.000000,8.000000,9.000000,9.000000


### Creazione del punto di ricerca
Per il momento prendiamo un dato a caso dal dataset come nostro candidato

In [1228]:
search = df.sample()
print(search)

       ID  age  longitude  latitude  attractiveness  sincerity  intelligence  \
193  6821   25    9.32285   45.2849               8          8             8   

     funniness  ambition  sports  ...  hiking_other  gaming_other  \
193          8         8       8  ...             9             5   

     clubbing_other  reading_other  tv_other  theater_other  movies_other  \
193               8              6         5              7             8   

     music_other  shopping_other  yoga_other  
193            8               8           8  

[1 rows x 45 columns]


Costruiamo un punto ad-hoc e salviamo i pesi

In [1229]:
search["age"] = search["age_other"]
search["sports"] = search["sports_other"]
search["tv_sports"] = search["tv_sports_other"]
search["exercise"] = search["exercise_other"]
search["dining"] = search["dining_other"]
search["hiking"] = search["hiking_other"]
search["gaming"] = search["gaming_other"]
search["clubbing"] = search["clubbing_other"]
search["reading"] = search["reading_other"]
search["tv"] = search["tv_other"]
search["theater"] = search["theater_other"]
search["movies"] = search["movies_other"]
search["music"] = search["music_other"]
search["shopping"] = search["shopping_other"]
search["yoga"] = search["yoga_other"]
importants = ['attractiveness_important', 'sincerity_important',
                  'intelligence_important', 'funniness_important',
                  'ambition_important']
weights = search[importants].to_numpy(dtype=np.uint8)[0]

In [1230]:
search = search.drop(columns=(list(df.filter(regex='.*other$')) + list(df.filter(regex='.*important$'))))
print(search)
print(search.columns)

       ID  age  longitude  latitude  attractiveness  sincerity  intelligence  \
193  6821   32    9.32285   45.2849               8          8             8   

     funniness  ambition  sports  ...  hiking  gaming  clubbing  reading  tv  \
193          8         8       8  ...       9       5         8        6   5   

     theater  movies  music  shopping  yoga  
193        7       8      8         8     8  

[1 rows x 24 columns]
Index(['ID', 'age', 'longitude', 'latitude', 'attractiveness', 'sincerity',
       'intelligence', 'funniness', 'ambition', 'sports', 'tv_sports',
       'exercise', 'dining', 'art', 'hiking', 'gaming', 'clubbing', 'reading',
       'tv', 'theater', 'movies', 'music', 'shopping', 'yoga'],
      dtype='object')


In [1213]:
search.iloc[:, 4:9] = 0
print(search)
print(weights)

      ID  age  longitude  latitude  attractiveness  sincerity  intelligence  \
88  3070   27    10.4549   44.5683               0          0             0   

    funniness  ambition  sports  ...  hiking  gaming  clubbing  reading  tv  \
88          0         0       4  ...       8       3         6        2   5   

    theater  movies  music  shopping  yoga  
88        7       2      6         5     8  

[1 rows x 24 columns]
[2 2 3 2 2]


Prepariamo il dataset alla ricerca.

Salviamo gli ID utenti per poter recuperare tutte le loro informazioni alla fine
Prepariamo il dataset alla ricerca. 

In [1231]:
df = df[df["ID"] != search.iloc[0, 0]]
IDs = df["ID"].to_list()
IDs.append(search.iloc[0,0])
IDs = np.array(IDs)
df = df.drop(columns=(["ID"] + list(df.filter(regex='.*other$')) + list(df.filter(regex='.*important$'))))
print(df.columns)
search = search.drop(columns=["ID"])
print(search.columns)

Index(['age', 'longitude', 'latitude', 'attractiveness', 'sincerity',
       'intelligence', 'funniness', 'ambition', 'sports', 'tv_sports',
       'exercise', 'dining', 'art', 'hiking', 'gaming', 'clubbing', 'reading',
       'tv', 'theater', 'movies', 'music', 'shopping', 'yoga'],
      dtype='object')
Index(['age', 'longitude', 'latitude', 'attractiveness', 'sincerity',
       'intelligence', 'funniness', 'ambition', 'sports', 'tv_sports',
       'exercise', 'dining', 'art', 'hiking', 'gaming', 'clubbing', 'reading',
       'tv', 'theater', 'movies', 'music', 'shopping', 'yoga'],
      dtype='object')


Convertiamo tutto in numpy array per un'elaborazione numerica più facile, il punto di ricerca sarà l'ultimo punto

In [1232]:
search = search.to_numpy()[0]
X = df.to_numpy()
X = np.vstack((X,search))
print("X:",X, X.shape)

X: [[23.      10.1383  44.7255  ...  7.       9.       5.     ]
 [29.      10.5231  44.7531  ...  8.       6.       8.     ]
 [26.       9.97214 44.9533  ...  8.       7.       6.     ]
 ...
 [28.       9.38892 44.9412  ...  8.       8.       7.     ]
 [24.      10.8498  45.2058  ...  7.       8.       7.     ]
 [32.       9.32285 45.2849  ...  8.       8.       8.     ]] (234, 23)


## Clustering

In [1234]:
clustering = HDBSCAN(min_cluster_size=6, n_jobs=-1, store_centers="centroid", allow_single_cluster=True)
clustering.fit(X[:, np.r_[5:8, 9:23]])

# ri-assegnamo i punti rumorosi
centroids = clustering.centroids_
print("Centroids shape:", centroids.shape)
noisy = X[clustering.labels_ == -1]
noisy = noisy[:,np.r_[5:8, 9:23]]
print("Noisy points shape:",noisy.shape)
distmat = cdist(noisy, centroids)
idxs = np.argmin(distmat, axis=1)
clustering.labels_[clustering.labels_ == -1] = idxs

# filtriamo
print("X before:",X.shape)
X = X[clustering.labels_ == clustering.labels_[-1]]
IDs = IDs[clustering.labels_ == clustering.labels_[-1]]
print("X after:",X.shape)
print(X)

Centroids shape: (1, 17)
Noisy points shape: (217, 17)
X before: (234, 23)
X after: (234, 23)
[[23.      10.1383  44.7255  ...  7.       9.       5.     ]
 [29.      10.5231  44.7531  ...  8.       6.       8.     ]
 [26.       9.97214 44.9533  ...  8.       7.       6.     ]
 ...
 [28.       9.38892 44.9412  ...  8.       8.       7.     ]
 [24.      10.8498  45.2058  ...  7.       8.       7.     ]
 [32.       9.32285 45.2849  ...  8.       8.       8.     ]]


## Deformazione delle variabili
Facciamo valere le preferenze

In [1235]:
exps = [(1/5), (1/1.3), 1, 2, 5]
for cnt, i in enumerate(range(4,9)):
    X[:-1,i] = np.int64(np.float_power((11-X[:-1,i]), exps[weights[cnt]-1]))
print(X)

[[23.      10.1383  44.7255  ...  7.       9.       5.     ]
 [29.      10.5231  44.7531  ...  8.       6.       8.     ]
 [26.       9.97214 44.9533  ...  8.       7.       6.     ]
 ...
 [28.       9.38892 44.9412  ...  8.       8.       7.     ]
 [24.      10.8498  45.2058  ...  7.       8.       7.     ]
 [32.       9.32285 45.2849  ...  8.       8.       8.     ]]


## Nearest Neighbor
È arrivato il momento tanto atteso di trovare i punti più simili a quello di ricerca

In [1236]:
nn = NearestNeighbors(n_jobs=-1)
nn.fit(X)

dsts, idxs = nn.kneighbors(X[-1].reshape(1,-1), return_distance=True, n_neighbors=6)
print("Indici:",idxs)
print("Distanze:",dsts)
print(X[idxs])

Indici: [[233 221 180 111  26 217]]
Distanze: [[ 0.         11.28769415 12.04218239 12.14874869 12.18324477 12.2131312 ]]
[[[32.       9.32285 45.2849   8.       8.       8.       8.
    8.       8.       7.       7.       8.       8.       9.
    5.       8.       6.       5.       7.       8.       8.
    8.       8.     ]
  [34.      10.4951  45.0903   6.       1.       4.       5.
    8.       5.       7.       6.       7.       7.       5.
    2.       7.       5.       4.       7.       7.       7.
    7.       7.     ]
  [35.       9.40549 45.1993   8.       1.       3.       5.
    5.       4.       4.       7.       8.       7.       7.
    3.       8.       6.       7.       7.       7.       7.
    6.       8.     ]
  [31.      10.0117  44.942    8.       1.       3.       5.
    5.       3.       5.       6.       8.       6.       7.
    6.       8.       6.       5.       7.       6.       7.
    5.       7.     ]
  [33.       9.76509 44.1732   7.       1.       3.       

## Risultato

In [1237]:
with pd.option_context('display.max_columns', None):
    print("ID punto di ricerca:",IDs[-1])
    print("ID dei punti simili:",IDs[idxs[0, 1:]])
    print("Punto di ricerca:",df1[df1["ID"] == IDs[-1]])
    for i in range(1, idxs.shape[1]):
        print(f"{i}° punto più simile:",df1[df1["ID"] == IDs[idxs[0,i]]])

ID punto di ricerca: 6821
ID dei punti simili: [8280 6427 3720  995 8186]
Punto di ricerca:        ID       name                         bio  gender  age  longitude  \
193  6821  Name 6821  Lorem ipsum dolor sit amet       2   25    9.32285   

     latitude  attractiveness  sincerity  intelligence  funniness  ambition  \
193   45.2849               8          8             8          8         8   

     sports  tv_sports  exercise  dining  art  hiking  gaming  clubbing  \
193       8          7         7       8    8       9       5         8   

     reading  tv  theater  movies  music  shopping  yoga  gender_other  \
193        6   5        7       8      8         8     8             2   

     age_other  age_flag_other  age_radius_other  distance_flag_other  \
193         32               1                17                    1   

     distance_km_other  attractiveness_important  sincerity_important  \
193                 30                         1                    2   

  

In [1238]:
dsts = pdist(X[idxs[0]], metric="cosine")[0:5]
print((2-dsts)/2*100)

[99.26522258 99.15160219 99.18836979 99.18299623 99.16754164]
